18762 Circuit Simulation & Optimization Methods  
Lucas Godshalk  
Spring 2022  

# Project 3 - Power Flow Infeasibility Analysis

## Infeasibility Analysis Overview

AC power flow simulations fundamentally have the goal of converging on a physical solution for a given power network. Improvements in formulating these simulators such as utilizing IV formulation or the inclusion of Tx-stepping have helped increas the robustness of these simulators. This makes them capable of solving increasingly difficult and large networks. But despite these improvements, they do not provide a guarantee that power flow will converge to a physical solution.

This presents a challenge when a simulator is presented with an infeasible network. An infeasible network, by definition, has no valid solution for AC power flow. If power flow had a globally robust method, we could at least categorically define these infeasible networks. Unfortunately, because this is not the case, it introduces uncertainty into power flow analysis. Answering the question "Is the network infeasible, or was the simulator not robust enough?" is difficult as the system has no method to differentiate the two. Infeasibility analysis, as the name implies, sets out to solve this problem. And in formulating the problem, ideally we should know not only if a network is infeasible, but also where the infeasibility exists on the network and its magnitude.

When using IV formulation, we can define infeasibility quite simply: a lack of real or imaginary current at one or more buses. Theoretically, a sufficient injection of current at the correct set of nodes would allow any network to have at a physical solution. If the sum of all of these _infeasibility currents_ is greater than zero, then we know that we are simulating an infeasible network. But while we can imagine a network where an unlimited amount of real and imaginary current is allowed to flow at every bus (in a similar manner to the slack bus) in order to create this solution, we need to ensure that no more of this infeasibility current is allowed to flow than is absolutely necessary. This requirement of minimizing the infeasibility current cannot be solved using standard power flow formulation alone, instead it requires solving an optimization problem. 

When formulating an optimization problem, we start by defining our objective function. In this case, given a vector of infeasibility currents, our goal is to minimize the $\ell^2$ norm of this vector. In this case, the $\ell^2$ norm can be stated as $\sqrt{\sum_{x=0}^{n} (I_{r,x}^2 + I_{i,x}^2)}$, which creates an incentive to keep any individual value of $I$ as low as possible. 

Next, we need to define the equality and inequality constraints that should govern our optimization problem. Fortunately, our equality constraints are already defined in the form of our power flow equations. In this particular formulation, no inequality constraints exist, although they could if we were to include constraints like $Q_{min}$ or $Q_{max}$ for a generator. Unfortunately, this introduces the first wrinkle, which is that the power flow equations are non-linear, which means our equality constraints are not affine. In optimization, this means our problem is non-convex and has several possible local minima in addition to the global minimum. 

### Derivations

Below are all of the $\frac{\partial{\mathcal{L}}}{\partial{X_x}}$ derivations for each component of the original power flow simulator. The $\lambda_x$ derivatives are omitted as they match the original derivations from project 2. All symbolic derivations are performed using sympy. For loads and generators, symbol naming matches variable names in the codebase, so the appearance may appear somewhat strange in their latex format.

In [1]:
from sympy import *
import numpy as np
import IPython.display as disp

def lagragify_and_display_derivatives(variables, lambdas, eqns):
    lagrange = np.dot(lambdas, eqns)
    display_derivatives(lagrange, variables)

def display_derivatives(lagrange, derivatives):
    print("Lagrange:")
    disp.display(lagrange)

    print("Derivatives:")

    for derivative in derivatives:
        print(f'd{derivative}:')
        disp.display(diff(lagrange, derivative))

#### Slack

In [2]:
[Vrset, Viset] = symbols("Vrset Viset")

variables = [Vr, Vi, Isr, Isi] = symbols("Vr Vi I_Sr I_Si")

lambdas = [Lr, Li, Lsr, Lsi] = symbols("lambda_Vr lambda_Vi lambda_Sr lambda_Si")

eqns = [
    Isr,
    Isi,
    Vr - Vrset,
    Vi - Viset,
]

lagragify_and_display_derivatives(variables, lambdas, eqns)


Lagrange:


I_Si*lambda_Vi + I_Sr*lambda_Vr + lambda_Si*(Vi - Viset) + lambda_Sr*(Vr - Vrset)

Derivatives:
dVr:


lambda_Sr

dVi:


lambda_Si

dI_Sr:


lambda_Vr

dI_Si:


lambda_Vi

#### Infeasibility

In [3]:
variables = [Iir, Iii] = symbols("Iir Iii")

lambdas = [Lr, Li] = symbols("lambda_Vr lambda_Vi")

lagrange = Iir ** 2 + Iii ** 2 + Iir * Lr + Iii * Li

display_derivatives(lagrange, variables)

Lagrange:


Iii**2 + Iii*lambda_Vi + Iir**2 + Iir*lambda_Vr

Derivatives:
dIir:


2*Iir + lambda_Vr

dIii:


2*Iii + lambda_Vi

#### Branch

In [4]:
G, B = symbols('G B')
variables = [Vr_from, Vi_from, Vr_to, Vi_to] = symbols('V_from\,r V_from\,i V_to\,r V_to\,i')
lambdas = [Lr_from, Li_from, Lr_to, Li_to] = symbols('lambda_from\,r lambda_from\,i lambda_to\,r lambda_to\,i')

eqns = [
    G * Vr_from - G * Vr_to + B * Vi_from - B * Vi_to,
    G * Vi_from - G * Vi_to - B * Vr_from + B * Vr_to,
    G * Vr_to - G * Vr_from + B * Vi_to - B * Vi_from,
    G * Vi_to - G * Vi_from - B * Vr_to + B * Vr_from   
]

lagragify_and_display_derivatives(variables, lambdas, eqns)

Lagrange:


lambda_from,i*(-B*V_from,r + B*V_to,r + G*V_from,i - G*V_to,i) + lambda_from,r*(B*V_from,i - B*V_to,i + G*V_from,r - G*V_to,r) + lambda_to,i*(B*V_from,r - B*V_to,r - G*V_from,i + G*V_to,i) + lambda_to,r*(-B*V_from,i + B*V_to,i - G*V_from,r + G*V_to,r)

Derivatives:
dV_from,r:


-B*lambda_from,i + B*lambda_to,i + G*lambda_from,r - G*lambda_to,r

dV_from,i:


B*lambda_from,r - B*lambda_to,r + G*lambda_from,i - G*lambda_to,i

dV_to,r:


B*lambda_from,i - B*lambda_to,i - G*lambda_from,r + G*lambda_to,r

dV_to,i:


-B*lambda_from,r + B*lambda_to,r - G*lambda_from,i + G*lambda_to,i

#### Transformer

(transformer losses are omitted since they utilize the branch stamps)

In [5]:

trcos, trsin = symbols('trcos trsin')
variables = [Vr, Vi, Ir, Ii, V_pri_r, V_pri_i] = symbols('V_r V_i I_pri\,r I_pri\,i V_sec\,r V_sec\,i')
lambdas = [Lr, Li, Lir, Lii, Lvr, Lvi] = symbols('lambda_r lambda_i lambda_pri\,Ir lambda_pri\,Ii lambda_sec\,Vr lambda_sec\,Vi')

eqns = [
    Ir,
    Ii,
    Vr - trcos * V_pri_r + trsin * V_pri_i,
    Vi - trcos * V_pri_i - trsin * V_pri_r,
    -trcos * Ir - trsin * Ii,
    -trcos * Ii + trsin * Ir
]

lagragify_and_display_derivatives(variables, lambdas, eqns)


Lagrange:


I_pri,i*lambda_i + I_pri,r*lambda_r + lambda_pri,Ii*(V_i - V_sec,i*trcos - V_sec,r*trsin) + lambda_pri,Ir*(V_r + V_sec,i*trsin - V_sec,r*trcos) + lambda_sec,Vi*(-I_pri,i*trcos + I_pri,r*trsin) + lambda_sec,Vr*(-I_pri,i*trsin - I_pri,r*trcos)

Derivatives:
dV_r:


lambda_pri,Ir

dV_i:


...

lambda_i - lambda_sec,Vi*trcos - lambda_sec,Vr*trsin

dV_sec,r:


-lambda_pri,Ii*trsin - lambda_pri,Ir*trcos

dV_sec,i:


-lambda_pri,Ii*trcos + lambda_pri,Ir*trsin

#### Shunt

In [6]:
G, B = symbols('G B')
variables = [Vr, Vi] = symbols('Vr Vi')
lambdas = [Lr, Li] = symbols('lambda_r lambda_i')

eqns = [
    G * Vr - B * Vi,
    G * Vi + B * Vr
]

lagragify_and_display_derivatives(variables, lambdas, eqns)

Lagrange:


lambda_i*(B*Vr + G*Vi) + lambda_r*(-B*Vi + G*Vr)

Derivatives:
dVr:


B*lambda_i + G*lambda_r

dVi:


-B*lambda_r + G*lambda_i

#### Load

In [7]:

P, Q = symbols('self.P self.Q')
variables = [Vr, Vi] = symbols('V_r V_i')
lambdas = [Lr, Li] = symbols('lambda_r lambda_i')

Ir = (P * Vr + Q * Vi) / (Vr ** 2 + Vi ** 2)
Ii = (P * Vi - Q * Vr) / (Vr ** 2 + Vi ** 2)

lagrange = Lr * Ir + Li * Ii

print("Lagrange:")
disp.display(lagrange)

Lagrange:


lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2) + lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)

In [8]:
dVr = diff(lagrange, Vr)
print("dVr:\n")
disp.display(dVr)
print(dVr)
print("\n")

for variable in variables + lambdas:
    print(f'dVr_{variable}:\n')
    disp.display(diff(dVr, variable))
    print(diff(dVr, variable))
    print("\n")

dVr:



-2*V_r*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 - lambda_i*self.Q/(V_i**2 + V_r**2) + lambda_r*self.P/(V_i**2 + V_r**2)

-2*V_r*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 - lambda_i*self.Q/(V_i**2 + V_r**2) + lambda_r*self.P/(V_i**2 + V_r**2)


dVr_V_r:



8*V_r**2*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**3 + 8*V_r**2*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**3 + 4*V_r*lambda_i*self.Q/(V_i**2 + V_r**2)**2 - 4*V_r*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - 2*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2

8*V_r**2*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**3 + 8*V_r**2*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**3 + 4*V_r*lambda_i*self.Q/(V_i**2 + V_r**2)**2 - 4*V_r*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - 2*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2


dVr_V_i:



...

8*V_i*V_r*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**3 + 8*V_i*V_r*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**3 + 2*V_i*lambda_i*self.Q/(V_i**2 + V_r**2)**2 - 2*V_i*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_i*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_r*self.Q/(V_i**2 + V_r**2)**2


dVr_lambda_r:



-2*V_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 + self.P/(V_i**2 + V_r**2)

-2*V_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 + self.P/(V_i**2 + V_r**2)


dVr_lambda_i:



-2*V_r*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - self.Q/(V_i**2 + V_r**2)

-2*V_r*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - self.Q/(V_i**2 + V_r**2)




In [9]:
dVi = diff(lagrange, Vi)
print("dVi:\n")
disp.display(dVi)
print(dVi)
print("\n")

for variable in variables + lambdas:
    print(f'dVi_{variable}:\n')
    disp.display(diff(dVi, variable))
    print(diff(dVi, variable))
    print("\n")

dVi:



-2*V_i*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - 2*V_i*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 + lambda_i*self.P/(V_i**2 + V_r**2) + lambda_r*self.Q/(V_i**2 + V_r**2)

-2*V_i*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - 2*V_i*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 + lambda_i*self.P/(V_i**2 + V_r**2) + lambda_r*self.Q/(V_i**2 + V_r**2)


dVi_V_r:



8*V_i*V_r*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**3 + 8*V_i*V_r*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**3 + 2*V_i*lambda_i*self.Q/(V_i**2 + V_r**2)**2 - 2*V_i*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_i*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_r*self.Q/(V_i**2 + V_r**2)**2

8*V_i*V_r*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**3 + 8*V_i*V_r*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**3 + 2*V_i*lambda_i*self.Q/(V_i**2 + V_r**2)**2 - 2*V_i*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_i*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_r*self.Q/(V_i**2 + V_r**2)**2


dVi_V_i:



...

8*V_i**2*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**3 + 8*V_i**2*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**3 - 4*V_i*lambda_i*self.P/(V_i**2 + V_r**2)**2 - 4*V_i*lambda_r*self.Q/(V_i**2 + V_r**2)**2 - 2*lambda_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 - 2*lambda_r*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2


dVi_lambda_r:



-2*V_i*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 + self.Q/(V_i**2 + V_r**2)

-2*V_i*(V_i*self.Q + V_r*self.P)/(V_i**2 + V_r**2)**2 + self.Q/(V_i**2 + V_r**2)


dVi_lambda_i:



-2*V_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 + self.P/(V_i**2 + V_r**2)

-2*V_i*(V_i*self.P - V_r*self.Q)/(V_i**2 + V_r**2)**2 + self.P/(V_i**2 + V_r**2)




#### Generator

In [10]:
P, Vset = symbols('self.P self.Vset')
variables = [Vr, Vi, Q] = symbols('V_r V_i Q')
lambdas = [Lr, Li, LQ] = symbols('lambda_r lambda_i lambda_Q')

Ir = (P * Vr + Q * Vi) / (Vr ** 2 + Vi ** 2)
Ii = (P * Vi - Q * Vr) / (Vr ** 2 + Vi ** 2)
Q_k = Vset ** 2 - Vr ** 2 - Vi ** 2

lagrange = Lr * Ir + Li * Ii + Q_k * LQ

print("Lagrange:")
disp.display(lagrange)

Lagrange:


lambda_Q*(-V_i**2 - V_r**2 + self.Vset**2) + lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2) + lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)

In [11]:
dVr = diff(lagrange, Vr)
print("dVr:\n")
disp.display(dVr)
print(dVr)
print("\n")

for variable in variables + lambdas:
    print(f'dVr_{variable}:\n')
    disp.display(diff(dVr, variable))
    print(diff(dVr, variable))
    print("\n")

dVr:



-Q*lambda_i/(V_i**2 + V_r**2) - 2*V_r*lambda_Q - 2*V_r*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2 + lambda_r*self.P/(V_i**2 + V_r**2)

-Q*lambda_i/(V_i**2 + V_r**2) - 2*V_r*lambda_Q - 2*V_r*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2 + lambda_r*self.P/(V_i**2 + V_r**2)


dVr_V_r:



4*Q*V_r*lambda_i/(V_i**2 + V_r**2)**2 + 8*V_r**2*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**3 + 8*V_r**2*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**3 - 4*V_r*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*lambda_Q - 2*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 - 2*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2

4*Q*V_r*lambda_i/(V_i**2 + V_r**2)**2 + 8*V_r**2*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**3 + 8*V_r**2*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**3 - 4*V_r*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*lambda_Q - 2*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 - 2*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2


dVr_V_i:



...

-2*V_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2 + self.P/(V_i**2 + V_r**2)


dVr_lambda_i:



-Q/(V_i**2 + V_r**2) - 2*V_r*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2

-Q/(V_i**2 + V_r**2) - 2*V_r*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2


dVr_lambda_Q:



-2*V_r

-2*V_r




In [12]:
dVi = diff(lagrange, Vi)
print("dVi:\n")
disp.display(dVi)
print(dVi)
print("\n")

for variable in variables + lambdas:
    print(f'dVi_{variable}:\n')
    disp.display(diff(dVi, variable))
    print(diff(dVi, variable))
    print("\n")

dVi:



Q*lambda_r/(V_i**2 + V_r**2) - 2*V_i*lambda_Q - 2*V_i*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 - 2*V_i*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2 + lambda_i*self.P/(V_i**2 + V_r**2)

Q*lambda_r/(V_i**2 + V_r**2) - 2*V_i*lambda_Q - 2*V_i*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 - 2*V_i*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2 + lambda_i*self.P/(V_i**2 + V_r**2)


dVi_V_r:



2*Q*V_i*lambda_i/(V_i**2 + V_r**2)**2 - 2*Q*V_r*lambda_r/(V_i**2 + V_r**2)**2 + 8*V_i*V_r*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**3 + 8*V_i*V_r*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**3 - 2*V_i*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_i*self.P/(V_i**2 + V_r**2)**2

2*Q*V_i*lambda_i/(V_i**2 + V_r**2)**2 - 2*Q*V_r*lambda_r/(V_i**2 + V_r**2)**2 + 8*V_i*V_r*lambda_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**3 + 8*V_i*V_r*lambda_r*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**3 - 2*V_i*lambda_r*self.P/(V_i**2 + V_r**2)**2 - 2*V_r*lambda_i*self.P/(V_i**2 + V_r**2)**2


dVi_V_i:



...

Q/(V_i**2 + V_r**2) - 2*V_i*(Q*V_i + V_r*self.P)/(V_i**2 + V_r**2)**2


dVi_lambda_i:



-2*V_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 + self.P/(V_i**2 + V_r**2)

-2*V_i*(-Q*V_r + V_i*self.P)/(V_i**2 + V_r**2)**2 + self.P/(V_i**2 + V_r**2)


dVi_lambda_Q:



-2*V_i

-2*V_i




In [13]:
dQ = diff(lagrange, Q)
print("dQ:\n")
disp.display(dQ)
print(dQ)
print("\n")

for variable in variables + lambdas:
    print(f'dQ_{variable}:\n')
    disp.display(diff(dQ, variable))
    print(diff(dQ, variable))
    print("\n")

dQ:



V_i*lambda_r/(V_i**2 + V_r**2) - V_r*lambda_i/(V_i**2 + V_r**2)

V_i*lambda_r/(V_i**2 + V_r**2) - V_r*lambda_i/(V_i**2 + V_r**2)


dQ_V_r:



-2*V_i*V_r*lambda_r/(V_i**2 + V_r**2)**2 + 2*V_r**2*lambda_i/(V_i**2 + V_r**2)**2 - lambda_i/(V_i**2 + V_r**2)

-2*V_i*V_r*lambda_r/(V_i**2 + V_r**2)**2 + 2*V_r**2*lambda_i/(V_i**2 + V_r**2)**2 - lambda_i/(V_i**2 + V_r**2)


dQ_V_i:



...

V_i/(V_i**2 + V_r**2)


dQ_lambda_i:



-V_r/(V_i**2 + V_r**2)

-V_r/(V_i**2 + V_r**2)


dQ_lambda_Q:



0

0


